In [1]:
!pip install nltk

In [2]:
import nltk
import numpy as np
from nltk.corpus import stopwords
import string

In [3]:
import tensorflow_hub as hub

In [4]:
import pandas as pd

In [5]:
!kaggle datasets download -d ishikajohari/best-books-10k-multi-genre-data

Dataset URL: https://www.kaggle.com/datasets/ishikajohari/best-books-10k-multi-genre-data
License(s): CC0-1.0
  0% 0.00/4.63M [00:00<?, ?B/s]
100% 4.63M/4.63M [00:00<00:00, 84.7MB/s]


In [6]:
import zipfile

# Define the path to your zip file
file_path = '/content/best-books-10k-multi-genre-data.zip'  # Replace 'your_file.zip' with your file's name

# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/kaggle')  # Replace 'destination_folder' with your desired folder

In [7]:
embed = hub.load("https://www.kaggle.com/models/google/nnlm/TensorFlow2/en-dim128-with-normalization/1")

In [8]:
df = pd.read_csv("/content/drive/MyDrive/kaggle/goodreads_data.csv")
df = df[['Book', 'Author', 'Description', 'Genres']]

In [9]:
df.head(
)

,Book,Author,Description,Genres
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ..."
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',..."
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical..."
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp..."
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',..."


In [10]:
def cosine_similarity(vec1, vec2):
  v1 = vec1[0]
  v2 = vec2[0]
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [11]:
df['index'] = df.index
df.head()

,Book,Author,Description,Genres,index
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",0
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",1
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",2
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",3
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",4


In [12]:
!python -m nltk.downloader stopwords
stop_words = stopwords.words('english')

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
import re

In [14]:
df['description_cleaned'] = df['Description'].apply(lambda x: re.sub(r'[0-9]+', '', str(x)))
df['description_cleaned'] = df['description_cleaned'].apply(str.lower)
df['description_cleaned'] = df['description_cleaned'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['description_cleaned'] = df['description_cleaned'].apply(lambda x: ' '.join( w for w in x.split() if w not in stop_words))

In [15]:
df['Description'][0], df['description_cleaned'][0]

('The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.',
 'unforgettable novel childhood sleepy southern town crisis conscience rocked kill mockingbird became instant bestseller critical success first published went win pulitzer prize

In [16]:
description_embedding_dict = { i: embed([df['description_cleaned'][i]]).numpy() for i in df['index']}

In [17]:
#Locality Sensitive Hashing
#Making Planes
print(df.shape)
#About 10,000 values, want ~10 per hash bucket
#~1,000 hash buckets will be needed
#2^n = 1,000
#n = ~10

(10000, 6)


In [18]:
PLANES_NUM = 10
UNIVERSES_NUM = 10
DIMS = 128

In [88]:
np.random.seed(0)
all_planes = [np.random.normal(size=(DIMS, PLANES_NUM))
            for i in range(UNIVERSES_NUM)]

In [20]:
def get_hash_value(vec, planes):
  v = vec[0]
  dp = np.dot(v, planes)
  h = np.sign(dp) >= 0

  hash_val = 0
  for i in range(PLANES_NUM):
    hash_val += 2**i * h[i]
  return hash_val

In [21]:
idx_universe_dict = {i:np.zeros(UNIVERSES_NUM) for i in range(len(description_embedding_dict))}

In [22]:
hash_tables = []
for j in range(UNIVERSES_NUM):
  hash_table_ids = {i:[] for i in range(2**PLANES_NUM)}
  for i in range(len(description_embedding_dict)):
    hb = get_hash_value(description_embedding_dict.get(i), all_planes[j])
    idx_universe_dict.get(i)[j] = hb
    hash_table_ids[hb].append(i)
  hash_tables.append(hash_table_ids)
  for i in range(len(hash_table_ids)):
    if (len(hash_table_ids[i]) > 0 and len(hash_table_ids[i]) < 5):
      print(hash_table_ids[i])
      print("universe " + str(j) + "has uneven bucket sizes")
      break



[5813, 6201, 8530]
universe 0has uneven bucket sizes
[2172]
universe 1has uneven bucket sizes
[3080]
universe 2has uneven bucket sizes
[6946, 8247]
universe 3has uneven bucket sizes
[1392, 3219, 8916]
universe 4has uneven bucket sizes
[382, 8550, 8946]
universe 5has uneven bucket sizes
[1785, 8172]
universe 6has uneven bucket sizes
[1648, 7110, 7872, 8058]
universe 7has uneven bucket sizes
[6368, 7713, 8701, 9957]
universe 8has uneven bucket sizes
[7092]
universe 9has uneven bucket sizes


Since each universe has some hash buckets with less than 5 vectors, we will combine universes when finding similar vectors.

closest_vectors_indeces_test compares our vector to all the other vectors and is used as a comparison against the actual function we will use, closest_vectors_indeces, which only compares our vectors to vectors in the same hashbuckets. So far they have both achieved the same results, which means we won't miss any closest vectors by using closest_vectors_indeces.

In [23]:
# def closest_vectors_indeces_test(vec_idx, k = 5):
#   hash_vals = idx_universe_dict[vec_idx]
#   assert len(hash_vals) == 10
#   unique_vectors_idxs = set()

#   v = description_embedding_dict[vec_idx]
#   cos_similarity_vals = []

#   for i in range(len(description_embedding_dict)):
#     cos_similarity_vals.append(cosine_similarity(v, description_embedding_dict[i]))

#   cos_similarity_idxs = np.argsort(cos_similarity_vals)
#   cos_similarity_idxs = cos_similarity_idxs[::-1]
#   return cos_similarity_idxs[1:k + 1]



In [24]:
def closest_vectors_indeces(vec_idx, k = 5):
  hash_vals = idx_universe_dict[vec_idx]
  assert len(hash_vals) == 10
  unique_vectors_idxs = set()

  universe_iter = 0
  while(universe_iter < UNIVERSES_NUM):
    hash = hash_vals[universe_iter]
    try:
      new_vec_indexes = hash_tables[universe_iter].get(hash)
      for i in new_vec_indexes:
        if i != vec_idx:
          unique_vectors_idxs.add(i)
    except:
      print(str(hash), "for universe" , str(universe_iter), "not found")
    universe_iter += 1

  v = description_embedding_dict[vec_idx]
  cos_similarity_vals = []

  unique_vectors_idxs_l = list(unique_vectors_idxs)

  unique_vectors_idxs_l_filtered = match_genres(unique_vectors_idxs_l, vec_idx)
  for i in unique_vectors_idxs_l_filtered:
    cos_similarity_vals.append(cosine_similarity(v, description_embedding_dict[i]))

  cos_similarity_idxs = np.argsort(cos_similarity_vals)
  cos_similarity_idxs = cos_similarity_idxs[::-1]

  final_vecs = []
  for i in cos_similarity_idxs[:k]:
    final_vecs.append(unique_vectors_idxs_l_filtered[i])
  print(final_vecs)
  return final_vecs


This function performs further filtering of the vectors in the same hashbuckets as the original vector to make sure the recommended books are in the same genre.

In [25]:
def match_genres(ls, og_index):
  og_genres = df['Genres'][og_index][:4]
  filtered_ls = []
  for index in ls:
    top_genres = df['Genres'][index][:4]
    for i in top_genres:
      if (i in og_genres):
        filtered_ls.append(index)
        break
  return filtered_ls


In [26]:
#running the closest_vectors_indeces_test function


# book_recs_idxs = closest_vectors_indeces_test(4)
# print(len(book_recs_idxs))
# print("\n")
# print(df['Book'][4], df['description_cleaned'][4])
# for i in book_recs_idxs:
#   print(str(i), df['Book'][i], df['description_cleaned'][i])
#   print("\n")

In [66]:
def get_closest_books_by_index(ind):
  book_recs_idxs = closest_vectors_indeces(ind)
  # print(df['Book'][i], " ", df['description_cleaned'][ind])
  for i in book_recs_idxs:
    print(df['Book'][i], "by: ", df['Author'][i], "\nGenres: " , df['Genres'][i], "\nDescription: ",  df['Description'][i])
    print("\n")
  return

In [28]:
!pip install txtai
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


Creating a Search Tool for user to pick a book.

In [29]:
import txtai
import streamlit as st

In [30]:
book_titles = df.Book.values

In [31]:
ta_embeddings = txtai.Embeddings({
    'path': 'sentence-transformers/all-MiniLM-L6-v2'
})

ta_embeddings.index(book_titles)
ta_embeddings.save('ta_book_title_embeddings.gz')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [86]:
#insert searched book title
query = ""

In [87]:
book_titles = df.Book.values

result = ta_embeddings.search(query, 1)
# print(result)
# results_display = book_titles[result[0][0]]
# print(results_display)
# chosen_book_idx = df[df['Book'] == results_display]['index']
# print(chosen_book_idx)
print("Closest Book to query is: ", df['Book'][result[0][0]])
print("\n")
get_closest_books_by_index(result[0][0])

Closest Book to query is:  How to Win Friends and Influence People


[4170, 2310, 4289, 805, 3325]
The $100 Startup: Reinvent the Way You Make a Living, Do What You Love, and Create a New Future by:  Chris Guillebeau 
Genres:  ['Business', 'Nonfiction', 'Entrepreneurship', 'Self Help', 'Finance', 'Personal Development', 'Buisness'] 
Description:  Lead a life of adventure, meaning and purpose— and  earn a good living. “Thoughtful, funny, and compulsively readable, this guide shows how ordinary people can build solid livings, with independence and purpose, on their own terms.”—Gretchen Rubin, author of the #1  New York Times  bestseller  The Happiness Project Still in his early thirties, Chris Guillebeau completed a tour of every country on earth and yet he’s never held a “real job” or earned a regular paycheck. Rather, he has a special genius for turning ideas into income, and he uses what he earns both to support his life of adventure and to give back.  Chris identified 1,500 individua

In [ ]:
#######

In [ ]:
chosen_book_idx = -1

In [ ]:
# st.title('Book Recommendations')

# search = st.text_input('Search for a book: ', '')

# if st.button('Search'):
#   if search:
#     results = ta_embeddings.search(search, 10)

#     results_display = [book_titles[i] for i in results]

#     for result in results_display:
#       st.write(result)
#       if st.button('Pick'):
#         chosen_book_idx = df[df['Book'] == result]['index']
#         closest_book_idxs = get_closest_books_by_index(chosen_book_idx)
#         for i in closest_book_idxs:
#           st.write(df['Book'][i], df['Author'][i])



In [ ]:
def get_titles_and_embeddings():

  book_titles = df.Book.values

  ta_embeddings = txtai.Embeddings({
    'path': 'sentence-transformers/all-MiniLM-L6-v2'
  })
  ta_embeddings.load('ta_book_title_embeddings.gz')
  return book_titles, ta_embeddings


titles, embeddings = st.cache_data(get_titles_and_embeddings)()


st.title('Book Recommendations')

search = st.text_input('Search for a book: ', '')

if st.button('Search'):
  if search:
    results = ta_embeddings.search(search, 10)

    results_display = [book_titles[i] for i in results]

    for result in results_display:
      st.write(result)
      if st.button('Pick'):
        chosen_book_idx = df[df['Book'] == result]['index']
        closest_book_idxs = get_closest_books_by_index(chosen_book_idx)
        for i in closest_book_idxs:
          st.write(df['Book'][i], df['Author'][i])

2024-08-25 21:51:55.797 No runtime found, using MemoryCacheStorageManager
2024-08-25 21:51:55.808 No runtime found, using MemoryCacheStorageManager


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.80.84.214


In [ ]:
%%writefile recommender.py


Writing recommender.py


In [ ]:

! wget -q -O - ipv4.icanhazip.com


34.80.84.214


In [ ]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 4s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
! streamlit run bookrecommender.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.84.214:8501

your url is: https://sad-suits-visit.loca.lt
2024-08-26 00:58:57.042783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 00:58:57.126116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 00:58:57.176336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-26 00:59:02.768617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(10000, 6)
[5813, 620